In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv("learn.csv")

In [3]:
data.loanapply_insert_time = pd.to_datetime(data.loanapply_insert_time)
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)
data.birth_year = pd.to_datetime(data.birth_year)

In [4]:
del data["Unnamed: 0"]

In [5]:
x = data.copy()

In [6]:
x.loan_limit = np.log10(x.loan_limit + 1)
x.desired_amount = np.log10(x.desired_amount+1)
x.yearly_income = np.log10(x.yearly_income+1)
x.existing_loan_amt = np.log10(x.existing_loan_amt + 1)

In [7]:
c1 = pd.get_dummies(x.income_type)
c2 = pd.get_dummies(x.employment_type)
c2.rename(columns = {"기타":"기타_직업"}, inplace = True)
c3 = pd.get_dummies(x.houseown_type)
c4 = pd.get_dummies(x.purpose)
c4.rename(columns = {"기타": "기타_목적"}, inplace = True)

In [8]:
c = pd.concat([c1, c2, c3, c4], axis = 1)

In [9]:
x.drop(["income_type", "employment_type", "houseown_type", "purpose"], inplace = True, axis= 1)
x = pd.concat([x, c], axis = 1)

In [10]:
X_train = x[x['insert_time']<'2022-05-16']
X_test = x[x['insert_time']>= '2022-05-16']

In [11]:
Y_train = X_train['is_applied']
Y_test = X_test['is_applied']
del X_train['is_applied']
del X_test['is_applied']

In [12]:
k = data.is_applied.mean()

In [13]:
from catboost import CatBoostClassifier, Pool

In [14]:
train = Pool(
    data = X_train,
    label = Y_train,
    weight = Y_train * (1-2*k) + k
)

In [15]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [16]:
model.fit(train)

0:	learn: 0.5696149	total: 1.07s	remaining: 1.07s
1:	learn: 0.5437607	total: 1.9s	remaining: 0us


In [17]:
p = model.predict(X_test)

In [18]:
f1_score(Y_test, p)

0.20411632466128557